In [1]:
import ray
from ray.util.multiprocessing import Pool as RayPool

import bayesfast as bf
bf.utils.parallel.set_backend(RayPool(4))

2020-10-20 15:46:16,254	INFO services.py:1164 -- View the Ray dashboard at http://127.0.0.1:8265


In [2]:
def create_model():
    import tensorflow as tf
    
    @tf.function
    def model(x):
        return -1*(x*x)
    
    def grad(self,x):
        x  = tf.convert_to_tensor(x)
        with tf.GradientTape() as tape:
            tape.watch(x)
            res = model(x)
        return tape.gradient(res, [x])
    
    return model, grad

In [3]:
@ray.remote(num_gpus=1)
class Simulator(object):
    def __init__(self):
        self.model, self.grad = create_model()
        
    def fwd(self, x):
        return self.model(self.x)
    
    def gradient(self, x):
        return self.grad(self.x)

In [4]:
my_sim = Simulator.remote()

In [5]:
obj = my_sim.fwd.remote(0.1)

2020-10-20 15:46:37,528	WARNING worker.py:1072 -- The actor or task with ID ffffffffffffffff62223d8501000000 is pending and cannot currently be scheduled. It requires {CPU: 1.000000}, {GPU: 1.000000} for execution and {CPU: 1.000000}, {GPU: 1.000000} for placement, but this node only has remaining {node:128.55.224.191: 1.000000}, {accelerator_type:V100: 1.000000}, {GPU: 1.000000}, {memory: 243.261719 GiB}, {object_store_memory: 74.658203 GiB}. In total there are 0 pending tasks and 1 pending actors on this node. This is likely due to all cluster resources being claimed by actors. To resolve the issue, consider creating fewer actors or increase the resources available to this Ray cluster. You can ignore this message if this Ray cluster is expected to auto-scale.
(pid=4643) 2020-10-20 15:46:37,672	INFO (unknown file):0 -- gc.collect() freed 15 refs in 0.012681578984484076 seconds2020-10-20 15:46:37,716	INFO (unknown file):0 -- gc.collect() freed 82 refs in 0.056637575034983456 seconds

(

In [6]:
#nothing happens
ray.get(obj)

KeyboardInterrupt: 

In [7]:


bf.utils.random.set_generator(4)
den = bf.DensityLite(logp=my_sim.fwd.remote,grad=my_sim.gradient.remote,input_size=1)


In [9]:
sampling_params = dict(n_chain=1, n_iter=100, n_warmup=10, x_0=0.1,
                 random_generator=None, step_size=1e-6, adapt_step_size=False,
                 metric='diag', adapt_metric=False, max_change=1000.,
                 target_accept=0.6, gamma=0.05, k=0.75, t_0=10.,
                 initial_mean=None, initial_weight=1., adapt_window=60,
                 update_window=1, doubling=True)

In [10]:
foo = bf.sample(den,sample_trace=sampling_params)
foo

ValueError: failed to get finite logp and/or grad at x_0.

2020-10-20 15:47:47,938	INFO (unknown file):0 -- gc.collect() freed 57 refs in 0.11646380694583058 seconds
(pid=4643) 2020-10-20 15:47:47,933	INFO (unknown file):0 -- gc.collect() freed 158 refs in 0.0778966200305149 seconds
